UNData API Exercise
In this exercise, you'll redo the data gathering phase of the UNData Exploration project by using APIs instead of downloading csv files.

You'll make use of the World Bank Indicators API. Note that this API does not require an API key. Before attempting the exercise, it would be a good idea to skim through the Documentation page and to check out the Basic Call Structure article.

In [346]:
# Import libraries
import pandas as pd
import requests

1. Use the API to get all available data for the GDP per capita, PPP (constant 2017 international $) indicator. Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [347]:
# Set API endpoint
indicator = 'NY.GDP.PCAP.PP.KD'

endpoint = f'https://api.worldbank.org/v2/country/all/indicator/{indicator}?'

# Set params
# Use per_page=1000 to bring back 1000 rows per page instead of the default 50 rows to reduce API requests from 346 to 18
params ={
    'format' : 'json',
    'per_page' : 1000
}

# Request to API using GET
response = requests.get(endpoint, params=params)

# Check response
print(response)

<Response [200]>


In [348]:
# Save json response in a variable
gdpdata = response.json()

# Look at what gdpdata variable contains
print('API response (gdpdata)')
print(type(gdpdata))         # <class 'list'>
print(len(gdpdata))          # 2 (metadata + actual data)
print(gdpdata[0])            # metadata: page info, total rows
print(gdpdata[1][:2])        # first 2 rows of actual data

API response (gdpdata)
<class 'list'>
2
{'page': 1, 'pages': 18, 'per_page': 1000, 'total': 17290, 'sourceid': '2', 'lastupdated': '2025-10-07'}
[{'indicator': {'id': 'NY.GDP.PCAP.PP.KD', 'value': 'GDP per capita, PPP (constant 2021 international $)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2024', 'value': 3968.96375122681, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'NY.GDP.PCAP.PP.KD', 'value': 'GDP per capita, PPP (constant 2021 international $)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2023', 'value': 3948.14272105098, 'unit': '', 'obs_status': '', 'decimal': 0}]


In [349]:
# Get total number of pages
total_pages = gdpdata[0]['pages']
print(f'Total pages to fetch: {total_pages}')

Total pages to fetch: 18


In [352]:
# Create list to store all results
all_gdp_data = []

# Loop through all pages and collect data
for page in range(1, total_pages + 1):
    page_response = requests.get(f'{endpoint}&page={page}', params=params)
    page_data = page_response.json()
    all_gdp_data += page_data[1]

# Progress print: every 3 pages and on the last page
    if page % 3 == 0 or page == total_pages:
        print(f'Fetching page {page} of {total_pages}...')

# Final confirmation
print('All pages fetched successfully!')
print(f'Total records = 17290')
print(f'Records collected = {len(all_gdp_data)}')

Fetching page 3 of 18...
Fetching page 6 of 18...
Fetching page 9 of 18...
Fetching page 12 of 18...
Fetching page 15 of 18...
Fetching page 18 of 18...
All pages fetched successfully!
Total records = 17290
Records collected = 17290


In [ ]:
# Create an empty dataframe and include all data
gdp_df = pd.DataFrame(all_gdp_data)
print('DataFrame created and all data added')

print('Shape:', gdp_df.shape)
print(gdp_df.info())
gdp_df.head()

Empty DataFrame created and all data added
Shape: (17290, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17290 entries, 0 to 17289
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   indicator        17290 non-null  object 
 1   country          17290 non-null  object 
 2   countryiso3code  17290 non-null  object 
 3   date             17290 non-null  object 
 4   value            8461 non-null   float64
 5   unit             17290 non-null  object 
 6   obs_status       17290 non-null  object 
 7   decimal          17290 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 1.1+ MB
None


,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'NY.GDP.PCAP.PP.KD', 'value': 'GDP per ...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2024,3968.963751,,,0
1,"{'id': 'NY.GDP.PCAP.PP.KD', 'value': 'GDP per ...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2023,3948.142721,,,0
2,"{'id': 'NY.GDP.PCAP.PP.KD', 'value': 'GDP per ...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,3974.244214,,,0
3,"{'id': 'NY.GDP.PCAP.PP.KD', 'value': 'GDP per ...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,3933.580905,,,0
4,"{'id': 'NY.GDP.PCAP.PP.KD', 'value': 'GDP per ...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,3861.068816,,,0


In [354]:
# Convert the list of dictionaries to a normalized dataframe
gdp_df = pd.json_normalize(all_gdp_data)
# Check dataframe
print('DataFrame after normalization')
gdp_df

DataFrame after normalization


,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,3968.963751,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
1,AFE,2023,3948.142721,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
2,AFE,2022,3974.244214,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
3,AFE,2021,3933.580905,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
4,AFE,2020,3861.068816,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...,...
17285,ZWE,1964,NaN,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZW,Zimbabwe
17286,ZWE,1963,NaN,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZW,Zimbabwe
17287,ZWE,1962,NaN,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZW,Zimbabwe
17288,ZWE,1961,NaN,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZW,Zimbabwe


2. Now, use the API to get all available data for Life expectancy at birth, total (years). This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [355]:
# Set API endpoint for life expectancy at birth, total years
indicator = 'SP.DYN.LE00.IN'

# Use per_page=1000 to bring back 1000 rows per page instead of the default of 50 rows to reduce API requests from 346 to 18
endpoint = f'https://api.worldbank.org/v2/country/all/indicator/{indicator}?'

# Set params
# Use per_page=1000 to bring back 1000 rows per page instead of the default 50 rows to reduce API requests from 346 to 18
params ={
    'format' : 'json',
    'per_page' : 1000
}

# Request to API using GET
response = requests.get(endpoint, params=params)

# Check response
print(response)

<Response [200]>


In [356]:
# Save json response in variable
lifedata = response.json()

# Look at what lifedata variable contains
print('API response (lifedata)')
print(type(lifedata))         # <class 'list'>
print(len(lifedata))          # 2 (metadata + actual data)
print(lifedata[0])            # metadata: page info, total rows
print(lifedata[1][:2])        # first 2 rows of actual data

API response (lifedata)
<class 'list'>
2
{'page': 1, 'pages': 18, 'per_page': 1000, 'total': 17290, 'sourceid': '2', 'lastupdated': '2025-10-07'}
[{'indicator': {'id': 'SP.DYN.LE00.IN', 'value': 'Life expectancy at birth, total (years)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2024', 'value': None, 'unit': '', 'obs_status': '', 'decimal': 0}, {'indicator': {'id': 'SP.DYN.LE00.IN', 'value': 'Life expectancy at birth, total (years)'}, 'country': {'id': 'ZH', 'value': 'Africa Eastern and Southern'}, 'countryiso3code': 'AFE', 'date': '2023', 'value': 65.146290855385, 'unit': '', 'obs_status': '', 'decimal': 0}]


In [357]:
# Get total number of pages
total_pages = lifedata[0]['pages']
print(f'Total pages to fetch: {total_pages}')

Total pages to fetch: 18


In [358]:
# Create list to store all results
all_life_data = []

# Loop through all pages and collect data
for page in range(1, total_pages + 1):
    page_response = requests.get(f'{endpoint}&page={page}', params=params)
    page_data = page_response.json()
    all_life_data += page_data[1]

    # Progress print: every 3 pages and on the last page
    if page % 3 == 0 or page == total_pages:
        print(f'Fetching page {page} of {total_pages}...')

# Final confirmation
print('All pages fetched successfully!')
print(f'Total records = 17290')
print(f'Records collected = {len(all_life_data)}')

Fetching page 3 of 18...
Fetching page 6 of 18...
Fetching page 9 of 18...
Fetching page 12 of 18...
Fetching page 15 of 18...
Fetching page 18 of 18...
All pages fetched successfully!
Total records = 17290
Records collected = 17290


In [359]:
# Create an empty DataFrame and include all data
life_df = pd.DataFrame(all_life_data)

print('Empty DataFrame created and all data added')
print('Shape:', life_df.shape)
print(life_df.info())
life_df.head()

Empty DataFrame created and all data added
Shape: (17290, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17290 entries, 0 to 17289
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   indicator        17290 non-null  object 
 1   country          17290 non-null  object 
 2   countryiso3code  17290 non-null  object 
 3   date             17290 non-null  object 
 4   value            16926 non-null  float64
 5   unit             17290 non-null  object 
 6   obs_status       17290 non-null  object 
 7   decimal          17290 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 1.1+ MB
None


,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'SP.DYN.LE00.IN', 'value': 'Life expect...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2024,NaN,,,0
1,"{'id': 'SP.DYN.LE00.IN', 'value': 'Life expect...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2023,65.146291,,,0
2,"{'id': 'SP.DYN.LE00.IN', 'value': 'Life expect...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,64.487020,,,0
3,"{'id': 'SP.DYN.LE00.IN', 'value': 'Life expect...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,62.979999,,,0
4,"{'id': 'SP.DYN.LE00.IN', 'value': 'Life expect...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,63.766484,,,0


In [360]:
# Convert the list of dictionaries to a normalized dataframe
life_df = pd.json_normalize(all_life_data)

# Check DataFrame
print('DataFrame after normalization')
life_df

DataFrame after normalization


,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,NaN,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
1,AFE,2023,65.146291,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
2,AFE,2022,64.487020,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
3,AFE,2021,62.979999,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
4,AFE,2020,63.766484,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZH,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...,...
17285,ZWE,1964,55.431000,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZW,Zimbabwe
17286,ZWE,1963,54.942000,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZW,Zimbabwe
17287,ZWE,1962,54.453000,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZW,Zimbabwe
17288,ZWE,1961,53.966000,,,0,SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",ZW,Zimbabwe


3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [361]:
# Check columns in GDP DataFrame
print('GDP DataFrame columns:')
print(gdp_df.columns.tolist())

# Check columns in Life Expectancy DataFrame
print('\nLife Expectancy DataFrame columns:')
print(life_df.columns.tolist())

# Preview first few rows of each
print('\nGDP DataFrame head:')
print(gdp_df.head())

print('\nLife Expectancy DataFrame head:')
print(life_df.head())

GDP DataFrame columns:
['countryiso3code', 'date', 'value', 'unit', 'obs_status', 'decimal', 'indicator.id', 'indicator.value', 'country.id', 'country.value']

Life Expectancy DataFrame columns:
['countryiso3code', 'date', 'value', 'unit', 'obs_status', 'decimal', 'indicator.id', 'indicator.value', 'country.id', 'country.value']

GDP DataFrame head:
  countryiso3code  date        value unit obs_status  decimal  \
0             AFE  2024  3968.963751                        0   
1             AFE  2023  3948.142721                        0   
2             AFE  2022  3974.244214                        0   
3             AFE  2021  3933.580905                        0   
4             AFE  2020  3861.068816                        0   

        indicator.id                                    indicator.value  \
0  NY.GDP.PCAP.PP.KD  GDP per capita, PPP (constant 2021 internation...   
1  NY.GDP.PCAP.PP.KD  GDP per capita, PPP (constant 2021 internation...   
2  NY.GDP.PCAP.PP.KD  GDP per ca

In [362]:
# Rename 'value' to distinguish the indicators
gdp_df_col = gdp_df.rename(columns={'value': 'GDP_per_capita'})
life_df_col = life_df.rename(columns={'value': 'Life_expectancy'})

In [363]:
# Keep only necessary columns
gdp_df_final = gdp_df_col[['country.value', 'countryiso3code', 'date', 'GDP_per_capita']]
life_df_final = life_df_col[['countryiso3code', 'date', 'Life_expectancy']]

In [364]:
# Check gdp_df_final
print('GDP DataFrame:')
gdp_df_final

GDP DataFrame:


,country.value,countryiso3code,date,GDP_per_capita
0,Africa Eastern and Southern,AFE,2024,3968.963751
1,Africa Eastern and Southern,AFE,2023,3948.142721
2,Africa Eastern and Southern,AFE,2022,3974.244214
3,Africa Eastern and Southern,AFE,2021,3933.580905
4,Africa Eastern and Southern,AFE,2020,3861.068816
...,...,...,...,...
17285,Zimbabwe,ZWE,1964,NaN
17286,Zimbabwe,ZWE,1963,NaN
17287,Zimbabwe,ZWE,1962,NaN
17288,Zimbabwe,ZWE,1961,NaN


In [365]:
# Check life_df_final
print('Life Expectancy DataFrame:')
life_df_final

Life Expectancy DataFrame:


,countryiso3code,date,Life_expectancy
0,AFE,2024,NaN
1,AFE,2023,65.146291
2,AFE,2022,64.487020
3,AFE,2021,62.979999
4,AFE,2020,63.766484
...,...,...,...
17285,ZWE,1964,55.431000
17286,ZWE,1963,54.942000
17287,ZWE,1962,54.453000
17288,ZWE,1961,53.966000


In [366]:
# Check how many matching rows exist before merging
common_rows = pd.merge(
    gdp_df[['countryiso3code', 'date']],  # GDP key columns
    life_df[['countryiso3code', 'date']],  # Life Expectancy key columns
    on=['countryiso3code', 'date'],
    how='inner'
)

print(f"Number of rows that will be in the inner merge: {len(common_rows)}")

Number of rows that will be in the inner merge: 18590


In [367]:
# Merge the 2 dataframes
merged_df = pd.merge(gdp_df_final, life_df_final, on=['countryiso3code', 'date'], how='inner')

#Check the result
print(f"Total rows in merged DataFrame: {len(merged_df)}")
merged_df

Total rows in merged DataFrame: 18590


,country.value,countryiso3code,date,GDP_per_capita,Life_expectancy
0,Africa Eastern and Southern,AFE,2024,3968.963751,NaN
1,Africa Eastern and Southern,AFE,2023,3948.142721,65.146291
2,Africa Eastern and Southern,AFE,2022,3974.244214,64.487020
3,Africa Eastern and Southern,AFE,2021,3933.580905,62.979999
4,Africa Eastern and Southern,AFE,2020,3861.068816,63.766484
...,...,...,...,...,...
18585,Zimbabwe,ZWE,1964,NaN,55.431000
18586,Zimbabwe,ZWE,1963,NaN,54.942000
18587,Zimbabwe,ZWE,1962,NaN,54.453000
18588,Zimbabwe,ZWE,1961,NaN,53.966000


4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the Country API. Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [372]:
# Set API endpoint for countries
endpoint = f'https://api.worldbank.org/v2/country/?'

# Set params
# Use per_page=300 to bring back 300 rows per page instead of the default 50 rows to reduce API requests
params ={
    'format' : 'json',
    'per_page' : 300
}

# Request to API using GET
response = requests.get(endpoint, params=params)

# Check response
print(response)

<Response [200]>


In [373]:
# Save json response in variable
countrydata = response.json()

# Look at what countrydata variable contains
print('API response (countrydata)')
print(type(countrydata))   # <class 'list'>
print(len(countrydata))    # 2 (metadata + actual data)
print(countrydata[0])      # metadata: page info, total rows
print(countrydata[1][:2])  # first 2 row of actual data

API response (countrydata)
<class 'list'>
2
{'page': 1, 'pages': 1, 'per_page': '300', 'total': 296}
[{'id': 'ABW', 'iso2Code': 'AW', 'name': 'Aruba', 'region': {'id': 'LCN', 'iso2code': 'ZJ', 'value': 'Latin America & Caribbean '}, 'adminregion': {'id': '', 'iso2code': '', 'value': ''}, 'incomeLevel': {'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'}, 'lendingType': {'id': 'LNX', 'iso2code': 'XX', 'value': 'Not classified'}, 'capitalCity': 'Oranjestad', 'longitude': '-70.0167', 'latitude': '12.5167'}, {'id': 'AFE', 'iso2Code': 'ZH', 'name': 'Africa Eastern and Southern', 'region': {'id': 'NA', 'iso2code': 'NA', 'value': 'Aggregates'}, 'adminregion': {'id': '', 'iso2code': '', 'value': ''}, 'incomeLevel': {'id': 'NA', 'iso2code': 'NA', 'value': 'Aggregates'}, 'lendingType': {'id': '', 'iso2code': '', 'value': 'Aggregates'}, 'capitalCity': '', 'longitude': '', 'latitude': ''}]


In [374]:
# Get total number of pages
total_pages = countrydata[0]['pages']
print(f'Total pages to fetch: {total_pages}')

Total pages to fetch: 1


In [375]:
# Create a list to store all country data
all_country_data = []

# Loop through all pages and collect data
for page in range(1, total_pages + 1):
    page_response = requests.get(f'{endpoint}&page={page}', params=params)
    page_data = page_response.json()
    all_country_data += page_data[1]
    
    # Progress print: every 2 pages and at the end
    if page % 2 == 0 or page == total_pages:
        print(f'Fetching page {page} of {total_pages}...')

# Final confirmation
print("All country pages fetched successfully!")
print(f'Total records = 296')
print(f'Records collected = {len(all_country_data)}')

Fetching page 1 of 1...
All country pages fetched successfully!
Total records = 296
Records collected = 296


In [376]:
# Create DataFrame and include all country data
country_df = pd.DataFrame(all_country_data)

print('Empty DataFrame created and all data added')
print('Shape:', country_df.shape)
print(country_df.info())
country_df.head()

Empty DataFrame created and all data added
Shape: (296, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           296 non-null    object
 1   iso2Code     296 non-null    object
 2   name         296 non-null    object
 3   region       296 non-null    object
 4   adminregion  296 non-null    object
 5   incomeLevel  296 non-null    object
 6   lendingType  296 non-null    object
 7   capitalCity  296 non-null    object
 8   longitude    296 non-null    object
 9   latitude     296 non-null    object
dtypes: object(10)
memory usage: 23.3+ KB
None


,id,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
0,ABW,AW,Aruba,"{'id': 'LCN', 'iso2code': 'ZJ', 'value': 'Lati...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'HIC', 'iso2code': 'XD', 'value': 'High...","{'id': 'LNX', 'iso2code': 'XX', 'value': 'Not ...",Oranjestad,-70.0167,12.5167
1,AFE,ZH,Africa Eastern and Southern,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
2,AFG,AF,Afghanistan,"{'id': 'MEA', 'iso2code': 'ZQ', 'value': 'Midd...","{'id': 'MNA', 'iso2code': 'XQ', 'value': 'Midd...","{'id': 'LIC', 'iso2code': 'XM', 'value': 'Low ...","{'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'}",Kabul,69.1761,34.5228
3,AFR,A9,Africa,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,
4,AFW,ZI,Africa Western and Central,"{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': ''}","{'id': 'NA', 'iso2code': 'NA', 'value': 'Aggre...","{'id': '', 'iso2code': '', 'value': 'Aggregates'}",,,


In [377]:
# Normalize nested dictionaries
country_df = pd.json_normalize(all_country_data)

# Check DataFrame
print('DataFrame after normalization')
country_df

DataFrame after normalization


,id,iso2Code,name,capitalCity,longitude,latitude,region.id,region.iso2code,region.value,adminregion.id,adminregion.iso2code,adminregion.value,incomeLevel.id,incomeLevel.iso2code,incomeLevel.value,lendingType.id,lendingType.iso2code,lendingType.value
0,ABW,AW,Aruba,Oranjestad,-70.0167,12.5167,LCN,ZJ,Latin America & Caribbean,,,,HIC,XD,High income,LNX,XX,Not classified
1,AFE,ZH,Africa Eastern and Southern,,,,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates
2,AFG,AF,Afghanistan,Kabul,69.1761,34.5228,MEA,ZQ,"Middle East, North Africa, Afghanistan & Pakistan",MNA,XQ,"Middle East, North Africa, Afghanistan & Pakis...",LIC,XM,Low income,IDX,XI,IDA
3,AFR,A9,Africa,,,,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates
4,AFW,ZI,Africa Western and Central,,,,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,XZN,A5,Sub-Saharan Africa excluding South Africa and ...,,,,NA,NA,Aggregates,,,,NA,NA,Aggregates,,,Aggregates
292,YEM,YE,"Yemen, Rep.",Sana'a,44.2075,15.352,MEA,ZQ,"Middle East, North Africa, Afghanistan & Pakistan",MNA,XQ,"Middle East, North Africa, Afghanistan & Pakis...",LIC,XM,Low income,IDX,XI,IDA
293,ZAF,ZA,South Africa,Pretoria,28.1871,-25.746,SSF,ZG,Sub-Saharan Africa,SSA,ZF,Sub-Saharan Africa (excluding high income),UMC,XT,Upper middle income,IBD,XF,IBRD
294,ZMB,ZM,Zambia,Lusaka,28.2937,-15.3982,SSF,ZG,Sub-Saharan Africa,SSA,ZF,Sub-Saharan Africa (excluding high income),LMC,XN,Lower middle income,IDX,XI,IDA


In [378]:
# Keep useful columns from country_df
country_df_col = country_df[['id', 'name', 'region.value', 'incomeLevel.value', 'capitalCity', 'iso2Code']]
country_df_col

,id,name,region.value,incomeLevel.value,capitalCity,iso2Code
0,ABW,Aruba,Latin America & Caribbean,High income,Oranjestad,AW
1,AFE,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
2,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,AF
3,AFR,Africa,Aggregates,Aggregates,,A9
4,AFW,Africa Western and Central,Aggregates,Aggregates,,ZI
...,...,...,...,...,...,...
291,XZN,Sub-Saharan Africa excluding South Africa and ...,Aggregates,Aggregates,,A5
292,YEM,"Yemen, Rep.","Middle East, North Africa, Afghanistan & Pakistan",Low income,Sana'a,YE
293,ZAF,South Africa,Sub-Saharan Africa,Upper middle income,Pretoria,ZA
294,ZMB,Zambia,Sub-Saharan Africa,Lower middle income,Lusaka,ZM


In [379]:
# Rename 'id' to match 'countryiso3code' for merging
country_df_final = country_df_col.rename(columns={'id': 'countryiso3code'})
country_df_final

,countryiso3code,name,region.value,incomeLevel.value,capitalCity,iso2Code
0,ABW,Aruba,Latin America & Caribbean,High income,Oranjestad,AW
1,AFE,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
2,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,AF
3,AFR,Africa,Aggregates,Aggregates,,A9
4,AFW,Africa Western and Central,Aggregates,Aggregates,,ZI
...,...,...,...,...,...,...
291,XZN,Sub-Saharan Africa excluding South Africa and ...,Aggregates,Aggregates,,A5
292,YEM,"Yemen, Rep.","Middle East, North Africa, Afghanistan & Pakistan",Low income,Sana'a,YE
293,ZAF,South Africa,Sub-Saharan Africa,Upper middle income,Pretoria,ZA
294,ZMB,Zambia,Sub-Saharan Africa,Lower middle income,Lusaka,ZM


In [380]:
# Merge country data with life expectancy
country_life_df = pd.merge(life_df_final, country_df_final, on='countryiso3code', how='left')
country_life_df

,countryiso3code,date,Life_expectancy,name,region.value,incomeLevel.value,capitalCity,iso2Code
0,AFE,2024,NaN,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
1,AFE,2023,65.146291,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
2,AFE,2022,64.487020,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
3,AFE,2021,62.979999,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
4,AFE,2020,63.766484,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
...,...,...,...,...,...,...,...,...
17285,ZWE,1964,55.431000,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW
17286,ZWE,1963,54.942000,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW
17287,ZWE,1962,54.453000,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW
17288,ZWE,1961,53.966000,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW


In [381]:
# Merge GDP data with country/life dataframe
country_life_gdp_df = pd.merge(gdp_df_final, country_life_df, on=['countryiso3code', 'date'], how='inner')
country_life_gdp_df

,country.value,countryiso3code,date,GDP_per_capita,Life_expectancy,name,region.value,incomeLevel.value,capitalCity,iso2Code
0,Africa Eastern and Southern,AFE,2024,3968.963751,NaN,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
1,Africa Eastern and Southern,AFE,2023,3948.142721,65.146291,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
2,Africa Eastern and Southern,AFE,2022,3974.244214,64.487020,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
3,Africa Eastern and Southern,AFE,2021,3933.580905,62.979999,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
4,Africa Eastern and Southern,AFE,2020,3861.068816,63.766484,Africa Eastern and Southern,Aggregates,Aggregates,,ZH
...,...,...,...,...,...,...,...,...,...,...
18585,Zimbabwe,ZWE,1964,NaN,55.431000,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW
18586,Zimbabwe,ZWE,1963,NaN,54.942000,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW
18587,Zimbabwe,ZWE,1962,NaN,54.453000,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW
18588,Zimbabwe,ZWE,1961,NaN,53.966000,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW


In [382]:
# Drop rows where region is empty or equals "Aggregates"
country_life_gdp_agg = country_life_gdp_df[country_life_gdp_df['region.value'] != 'Aggregates']
country_life_gdp_merged = country_life_gdp_agg.dropna(subset=['region.value'])

print('Shape after removing regions:', country_life_gdp_merged.shape)
country_life_gdp_merged

Shape after removing regions: (14105, 10)


,country.value,countryiso3code,date,GDP_per_capita,Life_expectancy,name,region.value,incomeLevel.value,capitalCity,iso2Code
4485,Afghanistan,AFG,2024,NaN,NaN,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,AF
4486,Afghanistan,AFG,2023,1983.812620,66.035,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,AF
4487,Afghanistan,AFG,2022,1981.710168,65.617,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,AF
4488,Afghanistan,AFG,2021,2144.166570,60.417,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,AF
4489,Afghanistan,AFG,2020,2769.685745,61.454,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,AF
...,...,...,...,...,...,...,...,...,...,...
18585,Zimbabwe,ZWE,1964,NaN,55.431,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW
18586,Zimbabwe,ZWE,1963,NaN,54.942,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW
18587,Zimbabwe,ZWE,1962,NaN,54.453,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW
18588,Zimbabwe,ZWE,1961,NaN,53.966,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,ZW


In [383]:
# Clean up and reorder columns for a 'final boss' dataframe

# Inspect all columns
print('Columns before cleaning:')
print(country_life_gdp_merged.columns.tolist())

Columns before cleaning:
['country.value', 'countryiso3code', 'date', 'GDP_per_capita', 'Life_expectancy', 'name', 'region.value', 'incomeLevel.value', 'capitalCity', 'iso2Code']


In [384]:
# Decide which columns to keep
keep_cols = [
    'countryiso3code',
    'name',
    'region.value',
    'incomeLevel.value',
    'capitalCity',
    'date',
    'GDP_per_capita',
    'Life_expectancy'
]

# Keep only those columns
boss_df = country_life_gdp_merged[keep_cols]
boss_df

,countryiso3code,name,region.value,incomeLevel.value,capitalCity,date,GDP_per_capita,Life_expectancy
4485,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2024,NaN,NaN
4486,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2023,1983.812620,66.035
4487,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2022,1981.710168,65.617
4488,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2021,2144.166570,60.417
4489,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2020,2769.685745,61.454
...,...,...,...,...,...,...,...,...
18585,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,1964,NaN,55.431
18586,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,1963,NaN,54.942
18587,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,1962,NaN,54.453
18588,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,1961,NaN,53.966


In [385]:
# Rename columns for cleaner presentation
next_boss_df = boss_df.rename(columns={
    'countryiso3code': 'Country_Code',
    'name': 'Country_Name',
    'region.value': 'Region',
    'incomeLevel.value': 'Income_Level',
    'capitalCity': 'Capital_City',
    'date': 'Year',
})

# Check the result
print('Next Boss DataFrame preview:')
print('Shape:', next_boss_df.shape)
next_boss_df

Next Boss DataFrame preview:
Shape: (14105, 8)


,Country_Code,Country_Name,Region,Income_Level,Capital_City,Year,GDP_per_capita,Life_expectancy
4485,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2024,NaN,NaN
4486,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2023,1983.812620,66.035
4487,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2022,1981.710168,65.617
4488,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2021,2144.166570,60.417
4489,AFG,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",Low income,Kabul,2020,2769.685745,61.454
...,...,...,...,...,...,...,...,...
18585,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,1964,NaN,55.431
18586,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,1963,NaN,54.942
18587,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,1962,NaN,54.453
18588,ZWE,Zimbabwe,Sub-Saharan Africa,Lower middle income,Harare,1961,NaN,53.966


In [386]:
# Reorder columns for final presentation
final_boss_df = next_boss_df[[
    'Country_Code',
    'Year',
    'Capital_City',
    'Country_Name',
    'Region',
    'Life_expectancy',
    'GDP_per_capita',
    'Income_Level'
]]

print('Final Boss DataFrame!')
final_boss_df

Final Boss DataFrame!


,Country_Code,Year,Capital_City,Country_Name,Region,Life_expectancy,GDP_per_capita,Income_Level
4485,AFG,2024,Kabul,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",NaN,NaN,Low income
4486,AFG,2023,Kabul,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",66.035,1983.812620,Low income
4487,AFG,2022,Kabul,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",65.617,1981.710168,Low income
4488,AFG,2021,Kabul,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",60.417,2144.166570,Low income
4489,AFG,2020,Kabul,Afghanistan,"Middle East, North Africa, Afghanistan & Pakistan",61.454,2769.685745,Low income
...,...,...,...,...,...,...,...,...
18585,ZWE,1964,Harare,Zimbabwe,Sub-Saharan Africa,55.431,NaN,Lower middle income
18586,ZWE,1963,Harare,Zimbabwe,Sub-Saharan Africa,54.942,NaN,Lower middle income
18587,ZWE,1962,Harare,Zimbabwe,Sub-Saharan Africa,54.453,NaN,Lower middle income
18588,ZWE,1961,Harare,Zimbabwe,Sub-Saharan Africa,53.966,NaN,Lower middle income
